In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import MaskFormerFeatureExtractor, MaskFormerForInstanceSegmentation
import PIL
from PIL import Image
import requests
from transformers import pipeline
import datasets
from datasets import load_dataset
import os
import evaluate
import torch
import cv2
import json
import codecs

c:\Users\kuian\.conda\envs\master\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_folder = './WE3DS/images/'
annotation_folder = './WE3DS/annotations/segmentation/SegmentationLabel/'
# Define the paths to the images and annotations
all_image_names = np.array(os.listdir(image_folder))

In [3]:
plant_classification = {
    'void': 'void',
    'soil': 'soil',
    'broad bean': 'crop',
    'corn spurry': 'weed',
    'red-root amaranth': 'weed',
    'common buckwheat': 'crop',
    'pea': 'crop',
    'red fingergrass': 'weed',
    'common wild oat': 'weed',
    'cornflower': 'weed',
    'corn cockle': 'weed',
    'corn': 'crop',
    'milk thistle': 'weed',
    'rye brome': 'weed',
    'soybean': 'crop',
    'sunflower': 'crop',
    'narrow-leaved plantain': 'weed',
    'small-flower geranium': 'weed',
    'sugar beet': 'crop'
}

In [ ]:
def get_image_meta_filepath(plant_name):
    return './meta/' + plant_name + '_images.json'

In [4]:
def get_image_list_for_crop(crop_name):
    # Create an empty list to store the dataset
    image_list = []
    crop_image_names = json.load(codecs.open(get_image_meta_filepath(crop_name), 'r', 'utf-8-sig'))
    weed_image_names = json.load(codecs.open(get_image_meta_filepath('images_with_no_crops'), 'r', 'utf-8-sig'))
    image_names = crop_image_names + weed_image_names

    # Iterate over the image and annotation paths
    for image_name in crop_image_names:
        # Load the image and annotation using PIL
        image = Image.open(image_folder + image_name)
        annotation = Image.open(annotation_folder + image_name)
        
        # Create a dictionary entry for the dataseta
        entry = {'image': image, 'annotation': annotation}
        
        # Add the entry to the dataset
        image_list.append(entry)

    return image_list

In [5]:
def create_and_split_dataset_for_crop(crop_image_list):
    dataset = datasets.Dataset.from_list(crop_image_list)
    dataset = dataset.train_test_split(test_size=0.2)
    train_ds = dataset["train"]
    test_ds = dataset["test"]
    return train_ds, test_ds

In [6]:
broad_bean_image_list = get_image_list_for_crop('broad_bean')
broad_bean_image_list

[{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1144>,
  'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=1600x1144>},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1144>,
  'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=1600x1144>},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1144>,
  'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=1600x1144>},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1144>,
  'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=1600x1144>},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1144>,
  'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=1600x1144>},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1144>,
  'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=1600x1144>},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=

In [7]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\kuian\.conda\envs\master\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [8]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

In [9]:
metric = evaluate.load("mean_iou")

In [10]:
with open('./WE3DS/class_names.txt', 'r') as file:
    labels = [line.strip() for line in file]

ids = list(range(1, 20))

id2label = dict(zip(ids, labels))
label2id = dict(zip(labels, ids))

num_labels = len(labels)

print("id2label:", id2label)
print("label2id:", label2id)

id2label: {1: 'void', 2: 'soil', 3: 'broad bean', 4: 'corn spurry', 5: 'red-root amaranth', 6: 'common buckwheat', 7: 'pea', 8: 'red fingergrass', 9: 'common wild oat', 10: 'cornflower', 11: 'corn cockle', 12: 'corn', 13: 'milk thistle', 14: 'rye brome', 15: 'soybean', 16: 'sunflower', 17: 'narrow-leaved plantain', 18: 'small-flower geranium', 19: 'sugar beet'}
label2id: {'void': 1, 'soil': 2, 'broad bean': 3, 'corn spurry': 4, 'red-root amaranth': 5, 'common buckwheat': 6, 'pea': 7, 'red fingergrass': 8, 'common wild oat': 9, 'cornflower': 10, 'corn cockle': 11, 'corn': 12, 'milk thistle': 13, 'rye brome': 14, 'soybean': 15, 'sunflower': 16, 'narrow-leaved plantain': 17, 'small-flower geranium': 18, 'sugar beet': 19}


In [11]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [12]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_fuse.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.classifier.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.0.

In [13]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
)

In [14]:
def initialize_trainer(train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
    )
    return trainer

In [15]:
broad_bean_train_ds, broad_bean_test_ds = create_and_split_dataset_for_crop(broad_bean_image_list)
broad_bean_train_ds.set_transform(train_transforms)
broad_bean_test_ds.set_transform(train_transforms)

In [16]:
braod_bean_trainer = initialize_trainer(broad_bean_train_ds, broad_bean_test_ds)
braod_bean_trainer.train()

c:\Users\kuian\.conda\envs\master\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/4200 [00:04<4:40:15,  4.00s/it]

{'loss': 2.9203, 'learning_rate': 5.9985714285714285e-05, 'epoch': 0.01}


  0%|          | 2/4200 [00:08<4:48:05,  4.12s/it]

{'loss': 2.919, 'learning_rate': 5.997142857142857e-05, 'epoch': 0.02}


  0%|          | 3/4200 [00:12<4:55:45,  4.23s/it]

...

{'loss': 0.2805, 'learning_rate': 5.5171428571428574e-05, 'epoch': 4.02}


  8%|▊         | 339/4200 [1:01:16<4:15:40,  3.97s/it]

{'loss': 0.1959, 'learning_rate': 5.515714285714286e-05, 'epoch': 4.04}


  8%|▊         | 340/4200 [1:01:19<4:14:48,  3.96s/it]

{'loss': 0.1908, 'learning_rate': 5.514285714285714e-05, 'epoch': 4.05}


In [ ]:
braod_bean_trainer.save_model('broad_bean')

In [ ]:
# Interpret results
# Add method to rewrite labels in the annotions